# 07 - Interactive Demo Interface

This notebook creates an interactive demo interface using Gradio for showcasing the fine-tuned BRD extraction model.

## What we'll do:
1. Load the fine-tuned model and extractor
2. Create Gradio interface
3. Add sample BRDs for testing
4. Launch interactive demo
5. Optional: Deploy to Hugging Face Spaces

## 1. Setup and Imports

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from pydantic import ValidationError
import json
import sys
sys.path.append('../data/processed')
from brd_extractor import BRDExtractor, ProjectEstimation
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports successful")

## 2. Load Model and Initialize Extractor

In [ ]:
BASE_MODEL_ID = "meta-llama/Llama-3.2-1B"
FINETUNED_MODEL_DIR = "../models/final/llama-3.2-1b-brd-final"

print("Loading model for demo...\n")

# Quantization config
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

# Load model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)
model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL_DIR)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

# Initialize extractor
extractor = BRDExtractor(model, tokenizer)

print("✓ Model loaded and ready for demo")

## 3. Define Sample BRDs

In [ ]:
SAMPLE_BRDS = {
    "E-commerce Mobile App": """Business Requirements Document
Project: E-commerce Mobile Application

Overview:
We require a cross-platform mobile application (iOS and Android) for our e-commerce business. 
The app will feature product browsing, shopping cart functionality, secure checkout, 
order tracking, and user account management.

Technical Scope:
- React Native development
- Integration with existing REST API
- Payment gateway integration (Stripe)
- Push notifications
- Analytics integration

Resource Requirements:
- 2 senior mobile developers
- 1 UI/UX designer
- 1 QA engineer

Timeline: 
The project is estimated to take 16 weeks from kickoff to production deployment.

Effort Estimation:
Total development effort is estimated at 960 hours across all team members.

Budget:
The total project cost is projected at $120,000, including all development, 
design, testing, and deployment activities.""",
    
    "CRM System": """Business Requirements Document
Project: Customer Relationship Management System

We need a comprehensive CRM system to manage our sales pipeline, customer interactions, 
and reporting. The system should include lead management, opportunity tracking, 
email integration, and dashboard analytics.

The development team will consist of 3 full-stack developers and 1 business analyst.
Project duration is estimated at 12 weeks with a total effort of 720 hours.
Budget allocation is $90,000 for the complete solution.""",
    
    "Website Redesign": """Business Requirements Document
Project: Corporate Website Redesign

Objective: Complete redesign of our corporate website with modern UI/UX, 
improved performance, and integrated CMS for content management.

Key deliverables include:
- Responsive design for all devices
- WordPress CMS integration
- SEO optimization
- Contact forms and lead capture
- Blog functionality

Team: 2 frontend developers, 1 backend developer, 1 designer
Timeline: 10 weeks
Estimated hours: 600 hours total
Project cost: $75,000""",
    
    "API Development": """Business Requirements Document
Project: RESTful API Development for Healthcare System

We need to develop a secure RESTful API to connect our healthcare management system 
with third-party applications. The API must be HIPAA compliant and include 
authentication, rate limiting, and comprehensive documentation.

Features:
- Patient data endpoints
- Appointment scheduling
- Medical records access
- OAuth 2.0 authentication
- API documentation (Swagger)

Resources: 2 backend developers, 1 security specialist
Duration: 8 weeks
Effort: 480 hours
Budget: $60,000""",
    
    "Data Pipeline": """Business Requirements Document
Project: ETL Data Pipeline Implementation

Build a robust data pipeline to extract data from multiple sources (databases, APIs, files), 
transform it according to business rules, and load it into our data warehouse for analytics.

Technical Requirements:
- Apache Airflow orchestration
- Data validation and quality checks
- Error handling and logging
- Automated testing
- Monitoring and alerts

Team composition: 2 data engineers, 1 DevOps engineer
Project timeline: 14 weeks
Total effort: 840 hours
Total cost: $105,000"""
}

print(f"✓ Loaded {len(SAMPLE_BRDS)} sample BRDs")

## 4. Create Demo Functions

In [ ]:
def extract_from_brd(brd_text: str) -> tuple:
    """
    Extract project estimation from BRD text.
    Returns formatted JSON and status message.
    """
    if not brd_text.strip():
        return "Please enter a BRD document.", ""
    
    try:
        # Extract using the model
        result = extractor.extract(brd_text, validate=True)
        
        if result["success"] and result["validated"]:
            # Format output
            data = result["data"]
            formatted_output = f"""✓ Extraction Successful!

Effort Hours:    {data['effort_hours']:,.1f}
Timeline (Weeks): {data['timeline_weeks']}
Cost (USD):      ${data['cost_usd']:,.2f}

JSON Output:
{json.dumps(data, indent=2)}
"""
            status = "✓ Valid and Validated"
            return formatted_output, status
        
        elif result["success"] and not result["validated"]:
            formatted_output = f"""⚠ Extraction successful but validation failed.

Extracted Data:
{json.dumps(result['data'], indent=2)}

Validation Errors:
{chr(10).join(result['errors'])}
"""
            status = "⚠ Needs Validation Review"
            return formatted_output, status
        
        else:
            formatted_output = f"""✗ Extraction Failed

Errors:
{chr(10).join(result['errors'])}

Raw Model Output:
{result['raw_output']}
"""
            status = "✗ Extraction Failed"
            return formatted_output, status
    
    except Exception as e:
        return f"Error: {str(e)}", "✗ Error Occurred"

def load_sample(sample_name: str) -> str:
    """Load a sample BRD."""
    return SAMPLE_BRDS.get(sample_name, "")

print("✓ Demo functions defined")

## 5. Create Gradio Interface

In [ ]:
# Create Gradio interface
with gr.Blocks(title="BRD Project Estimation Extractor", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 📊 BRD Project Estimation Extractor
        
        Extract structured project estimations from Business Requirements Documents using a 
        fine-tuned Llama 3.2 1B model with Pydantic validation.
        
        ### What it extracts:
        - **Effort Hours**: Total project effort
        - **Timeline (Weeks)**: Project duration
        - **Cost (USD)**: Budget estimate
        
        ### How it works:
        1. Paste your BRD document (or select a sample)
        2. Click "Extract Estimation"
        3. Get validated JSON output
        """
    )
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Input BRD")
            
            # Sample selector
            sample_dropdown = gr.Dropdown(
                choices=list(SAMPLE_BRDS.keys()),
                label="Load Sample BRD",
                value=None
            )
            
            # Input text area
            input_text = gr.Textbox(
                label="Business Requirements Document",
                placeholder="Paste your BRD document here...",
                lines=15,
                max_lines=25
            )
            
            # Extract button
            extract_btn = gr.Button("🚀 Extract Estimation", variant="primary", size="lg")
        
        with gr.Column(scale=1):
            gr.Markdown("### 📊 Extracted Estimation")
            
            # Status
            status_text = gr.Textbox(
                label="Status",
                interactive=False,
                lines=1
            )
            
            # Output text area
            output_text = gr.Textbox(
                label="Results",
                placeholder="Extracted estimation will appear here...",
                lines=15,
                max_lines=25,
                interactive=False
            )
    
    # Info section
    gr.Markdown(
        """
        ---
        ### 🔬 Technical Details
        - **Model**: Llama 3.2 1B (Fine-tuned with QLoRA)
        - **Training**: 8-bit quantization on CPU
        - **Validation**: Pydantic schema with custom validators
        - **Techniques**: Low-Rank Adaptation (LoRA), Gradient Checkpointing
        
        ### 📖 About
        This is a demonstration of state-of-the-art fine-tuning techniques for structured 
        data extraction. The model was trained on 1,000+ synthetic BRD documents using 
        QLoRA for parameter-efficient fine-tuning.
        """
    )
    
    # Event handlers
    sample_dropdown.change(
        fn=load_sample,
        inputs=[sample_dropdown],
        outputs=[input_text]
    )
    
    extract_btn.click(
        fn=extract_from_brd,
        inputs=[input_text],
        outputs=[output_text, status_text]
    )

print("✓ Gradio interface created")

## 6. Launch Demo

In [ ]:
# Launch the demo
print("Launching demo...\n")

demo.launch(
    share=False,  # Set to True to create public link
    server_port=7860,
    server_name="127.0.0.1",
    show_error=True,
    quiet=False
)

print("\n✓ Demo is running!")
print("  Open your browser to: http://127.0.0.1:7860")
print("  Press Ctrl+C to stop the demo")

## 7. Optional: Export Demo for Deployment

In [ ]:
# Create standalone app file for deployment
app_code = '''"""Standalone Gradio App for BRD Extraction

Run with: python app.py
"""

import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import json
from brd_extractor import BRDExtractor

# Load model
print("Loading model...")
quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(base_model, "./models/final/llama-3.2-1b-brd-final")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token
extractor = BRDExtractor(model, tokenizer)
print("Model loaded!")

def extract_from_brd(brd_text):
    if not brd_text.strip():
        return "Please enter a BRD document.", ""
    result = extractor.extract(brd_text, validate=True)
    if result["success"] and result["validated"]:
        data = result["data"]
        output = f'''✓ Extraction Successful!

Effort Hours:     {data['effort_hours']:,.1f}
Timeline (Weeks): {data['timeline_weeks']}
Cost (USD):       ${data['cost_usd']:,.2f}

JSON Output:
{json.dumps(data, indent=2)}'''
        return output, "✓ Valid and Validated"
    return "Extraction failed", "✗ Failed"

# Create interface
with gr.Blocks(title="BRD Extractor", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# BRD Project Estimation Extractor")
    with gr.Row():
        input_text = gr.Textbox(label="BRD Document", lines=10)
        output_text = gr.Textbox(label="Results", lines=10)
    status = gr.Textbox(label="Status")
    btn = gr.Button("Extract")
    btn.click(extract_from_brd, inputs=[input_text], outputs=[output_text, status])

if __name__ == "__main__":
    demo.launch()
'''

with open("../app.py", "w") as f:
    f.write(app_code)

print("✓ Standalone app saved to: ../app.py")
print("\nTo run: python app.py")

## Summary

### What we've built:
- ✓ Interactive Gradio demo interface
- ✓ Sample BRDs for testing
- ✓ Real-time extraction with validation
- ✓ User-friendly UI with status indicators
- ✓ Standalone app for deployment

### Features:
- **Sample BRDs**: Pre-loaded examples for quick testing
- **Real-time Extraction**: Immediate results
- **Validation Status**: Clear feedback on data quality
- **JSON Output**: Structured, validated data
- **Error Handling**: Graceful failure messages

### Deployment Options:
1. **Local**: Run on localhost (current setup)
2. **Hugging Face Spaces**: Deploy with `demo.launch(share=True)`
3. **Docker**: Containerize the app
4. **Cloud**: Deploy to AWS/GCP/Azure
5. **API**: Wrap in FastAPI for REST endpoints

### Usage:
1. Select a sample BRD or paste your own
2. Click "Extract Estimation"
3. View validated JSON output
4. Share with stakeholders!

### Next Steps:
- Share the demo link with colleagues
- Collect feedback on accuracy
- Iterate on the model with new data
- Deploy to production environment

### Files Created:
- `app.py`: Standalone Gradio app

Congratulations! You've built a complete end-to-end ML project! 🎉